In [13]:
import pandas as pd
data = pd.read_excel('MWH06_Update13_NLP_SampleInsight.xlsx')

In [ ]:
import time
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

def cluster_names (name, lower_sim, upper_sim):
    '''Calculate name similarity for input name compared to all names
    from a list of names and cluster names that are more similar than the
    specified threshold.
    '''
    cluster=[name]
    for name2 in activity_names:
        if name2 not in names_clusters.keys():
            if (similar (name,name2) > lower_sim)\
            & (similar (name,name2) < upper_sim):
                cluster.append(name2)
    return cluster

def cluster_names_df (name, names, lower_sim, upper_sim):
    '''Calculate name similarity for input name compared to all names
    from a list of names and cluster names that are more similar than the
    specified threshold.
    
    test: 
    name = 'aaaa'
    namel = [name]
    names = ['eeaa', 'aaaa', 'aaab', 'aaac']
    cluster_names (name, names, 0.5 ,0.95)
    
    '''
    namel = [name]
    names_df = pd.DataFrame()
    names_df['query'] = [len(names)*namel][0]
    names_df['name'] = names
    names_df['distance'] = names_df.apply(lambda x:\
                                          similar(str(x['query']),str(x['name'])), axis=1)
    names_df = names_df[(names_df['distance']> lower_sim) & (names_df['distance'] < upper_sim)]
    #print(names_df)
    cluster = list(names_df['name'])
    return cluster


def df_info(df):
    cols = df.columns
    rows_count = len(df)
    results = []
    for col in cols:
        null_count = df[col].isna().sum()
        coverage = round((rows_count-null_count)/rows_count, 2)
        uniques = len(df[col].unique())
        perc_uniques = round(100*uniques/rows_count)
        results.append([col,coverage, uniques, perc_uniques])
    
    coverage_df = pd.DataFrame(results, columns = ['column', 'coverage', 'uniques', '%uniques'])
    coverage_df ['type'] = list(df.dtypes.values)
    coverage_df = coverage_df[coverage_df['coverage']>0].sort_values(by=['coverage'], ascending=False)
    
    return coverage_df


def one_hot_encode_categorical (dataset, exclude):
    
    
    '''
    Identify then encode categorical features using the one-hot method
    :params: 
    dataset: input dataset (pandas dataframe)
    exclude: column to exclude from encoding (list), such as excludes, targets or the entities classified)
    '''
    
    exclude_col = dataset[[exclude]]
    features = dataset.drop([exclude], axis=1)
    features_cat = features.select_dtypes(exclude=['number'])
    features_num = features.select_dtypes(include=['number'])
    features_cat_dummies = pd.get_dummies(features_cat)
    encoded_dataset = pd.concat ([exclude_col, features_num,features_cat_dummies],axis=1)
    return (encoded_dataset)

# Feature Selection  

In [14]:
# Filter level rows to keep only activity rows
activity_data = data[data['Level'].isna()].drop(['Level'], axis=1)

# Filter empty columns
empty_cols = []
for col in activity_data.columns:
    null_count = activity_data[col].isna().sum()
    if null_count == len(activity_data): empty_cols.append(col)
#print('empty columns:', empty_cols)
keep = [c for c in activity_data.columns if c not in empty_cols]
activity_data = activity_data[keep]

# Filter values with one uniuqe value
single_unique_cols = []
for col in activity_data.columns:
    uniques = len(activity_data[col].unique())
    if uniques == 1: single_unique_cols.append(col)
keep = [c for c in activity_data.columns if c not in single_unique_cols]
#print('single value columns:', single_unique_cols)
activity_data = activity_data[keep]

# Filter start and finish columns (Their values are reflected in duration )  
keep = [c for c in activity_data.columns if not any(t in c for t in ['Start', 'Finish', 'Date'])]
activity_data = activity_data[keep]

# Filter Successors, Predecessors -> First Successor/Predecessor  
successors_data, predecessors_data = list(activity_data['Successors']), list(activity_data['Predecessors'])
activity_data['First_Predecessor'] = [s.split(',')[0] if str(s)!='nan' else 'none' for s in predecessors_data]
activity_data['First_Successor'] = [s.split(',')[0] if str(s)!='nan' else 'none' for s in successors_data]
exclude = [c for c in activity_data.columns if\
           (any(t in c for t in ['Successor','Predecessor']) & ('First' not in c))]
keep = [c for c in activity_data.columns if c not in exclude]
activity_data = activity_data[keep]

print('rows/columns count=', activity_data.shape)
activity_data.head()

rows/columns count= (1350, 62)


,Activity Type,Activity ID,Activity Name,Original Duration,Remaining Duration,Schedule % Complete,Actual Duration,At Completion Duration,BL Project Duration,BL1 Duration,...,At Completion Labor Units,BL Project Labor Units,BL1 Labor Units,Budgeted Labor Units,Remaining Labor Units,Variance - BL Project Labor Units,Variance - BL1 Labor Units,MSP Activity ID,First_Predecessor,First_Successor
2,Start Milestone,MWH.06.M1000,Project Start,0,0,1.0,0,0,0,0,...,0,0,0,0,0,0,0,15.0,none,A1170
3,Finish Milestone,MWH.06.M1010,MWH06 Admin/COLO1 - LV3 Cx Complete,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,3.0,MWH06.C1.Cx3040,MWH06.C1.Cx4000
4,Finish Milestone,MWH.06.M2010,MWH06 COLO2 - LV3 Cx Complete,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,17.0,MWH06.C2.Cx3040,MWH06.C2.Cx4010
5,Finish Milestone,MWH.06.M3010,MWH06 COLO3 - LV3 Cx Complete,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,26.0,MWH06.C3.Cx3040,MWH06.C3.Cx4000
6,Finish Milestone,MWH.06.M4010,MWH06 COLO4 - LV3 Cx Complete,0,0,0.0,0,0,0,0,...,0,0,0,0,0,0,0,4.0,MWH06.S.1910,MWH06.C4.Cx4000


In [15]:
activity_info = df_info(activity_data)
activity_info.head(10)

,column,coverage,uniques,%uniques,type
0,Activity Type,1.0,4,0,object
29,Critical,1.0,2,0,object
1,Activity ID,1.0,1350,100,object
34,WBS,1.0,360,27,object
35,WBS Name,1.0,56,4,object
36,WBS Path,1.0,360,27,object
39,Activity Resource Assignment Count,1.0,4,0,int64
40,Actual Completed Activities,1.0,2,0,int64
41,Actual In-Progress Activities,1.0,2,0,int64
42,BL Project Completed Activities,1.0,2,0,int64


# Cluster Analysis: Activity Names  

## Names Frequency  

In [ ]:
entity = 'Activity Name'
activity_names = activity_data[entity].unique()
print('The dataset has {n} unique activity names'.format(n = len(activity_names)))
activity_names_counts = activity_data[entity].value_counts()
activity_names_counts = pd.DataFrame(list(zip(activity_names_counts.index, activity_names_counts.values)),\
                                    columns = [entity,'count'])
#clusters_counts = pd.DataFrame(list(zip(clusters_counts.index, clusters_counts.values)),\
#                              columns = ['cluster', 'count']).sort_values(by=['count'], ascending=False)

activity_names_counts.to_excel('activity_names_counts.xlsx', index=False)
print('Frequent names:')
print(activity_names_counts[0:50])

## Names cluster  
Use distance function to cluster activity names by similarity  

In [ ]:
# start = time.time()
# names_clusters = {}
# for activity_name in activity_names:
#     if activity_name not in names_clusters.keys():
#         cluster = cluster_names_df(activity_name, activity_names, 0.8 ,0.95)
#         if len(cluster)>1:
#             names_clusters[activity_name] = cluster
# #             print(90*'-')
# #             print(activity_name, cluster)
# end = time.time()
# duration = round(end-start)
# print('duration:', duration)

# Clusters Members
Result copied to distance_clusters_members.txt

In [21]:
start = time.time()
names_clusters = {}
for activity_name in activity_names:
    if activity_name not in names_clusters.keys():
        cluster = cluster_names(activity_name, 0.8 ,0.95)
        if len(cluster)>1:
            names_clusters[activity_name] = cluster
            print(90*'+')
            print('key name:', activity_name)#, cluster)
            print(90*'-')
            for c in cluster: print(c)
end = time.time()
duration = round(end-start)
print('duration:', duration)

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: MWH06 Admin/COLO1 - LV3 Cx Complete
------------------------------------------------------------------------------------------
MWH06 Admin/COLO1 - LV3 Cx Complete
MWH06 COLO2 - LV3 Cx Complete
MWH06 COLO3 - LV3 Cx Complete
MWH06 COLO4 - LV3 Cx Complete
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: MWH06 Admin/COLO1 - Fitout Summary
------------------------------------------------------------------------------------------
MWH06 Admin/COLO1 - Fitout Summary
MWH06 COLO2 - Fitout Summary
MWH06 COLO3 - Fitout Summary
MWH06 COLO4 - Fitout Summary
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Foundation Start - Admin/COLO 1
------------------------------------------------------------------------------------------
Foundation Start - Admin/COLO 1
Fit out Start - Admin/COLO 1
Foundation S

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Pad Ready for MER COLO-2 Cell-4
------------------------------------------------------------------------------------------
Pad Ready for MER COLO-2 Cell-4
Pad Ready for MER COLO-3 Cell-1
Pad Ready for MER COLO-3 Cell-2
Pad Ready for MER COLO-3 Cell-3
Pad Ready for MER COLO-4 Cell-1
Pad Ready for MER COLO-4 Cell-2
Pad Ready for MER COLO-4 Cell-3
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Pad Ready for MER COLO-2 Cell-3
------------------------------------------------------------------------------------------
Pad Ready for MER COLO-2 Cell-3
Pad Ready for MER COLO-3 Cell-1
Pad Ready for MER COLO-3 Cell-2
Pad Ready for MER COLO-3 Cell-4
Pad Ready for MER COLO-4 Cell-1
Pad Ready for MER COLO-4 Cell-2
Pad Ready for MER COLO-4 Cell-4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Pad

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: AHU - ROJ COLO-4 Cell- 2
------------------------------------------------------------------------------------------
AHU - ROJ COLO-4 Cell- 2
UPM - ROJ - COLO2- Cell-2
UPM - ROJ - COLO3- Cell-2
UPM - ROJ - COLO-4 Cell- 1
UPM - ROJ - COLO-4 Cell- 2
UPM - ROJ - COLO-4 Cell- 3
UPM - ROJ - COLO-4 Cell- 4
PDU ROJ COLO-4 Cell-2
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: AHU - ROJ COLO-4 Cell- 1
------------------------------------------------------------------------------------------
AHU - ROJ COLO-4 Cell- 1
UPM - ROJ - COLO2- Cell-1
UPM - ROJ - COLO3- Cell-1
UPM - ROJ - COLO-4 Cell- 1
UPM - ROJ - COLO-4 Cell- 2
UPM - ROJ - COLO-4 Cell- 3
UPM - ROJ - COLO-4 Cell- 4
PDU ROJ COLO-4 Cell-1
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: AHU - ROJ COLO-4 Cell- 3
--------------------------

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: UPM - ROJ - COLO-4 Cell- 1
------------------------------------------------------------------------------------------
UPM - ROJ - COLO-4 Cell- 1
PDU ROJ COLO-4 Cell-1
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: UPM - ROJ - COLO-4 Cell- 2
------------------------------------------------------------------------------------------
UPM - ROJ - COLO-4 Cell- 2
PDU ROJ COLO-4 Cell-2
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: UPM - ROJ - COLO-4 Cell- 3
------------------------------------------------------------------------------------------
UPM - ROJ - COLO-4 Cell- 3
PDU ROJ COLO-4 Cell-3
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: UPM - ROJ - COLO-4 Cell- 4
--------------------------------------------------------------------

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Generator ROJ COLO 1- Cell-4
------------------------------------------------------------------------------------------
Generator ROJ COLO 1- Cell-4
Generator ROJ COLO 2- Cell - 1
Generator ROJ COLO 2- Cell - 2
Generator ROJ COLO 2- Cell - 3
Generator ROJ COLO 2- Cell - 4
Generator ROJ COLO 3- Cell - 1
Generator ROJ COLO 3- Cell - 2
Generator ROJ COLO 3- Cell - 3
Generator ROJ COLO 3- Cell - 4
Generator ROJ COLO 4- Cell - 2
Generator ROJ COLO 4- Cell - 1
Generator ROJ COLO 4- Cell - 3
Generator ROJ COLO 4- Cell - 4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Generator ROJ COLO 2- Cell - 1
------------------------------------------------------------------------------------------
Generator ROJ COLO 2- Cell - 1
Generator ROJ COLO 3- Cell - 2
Generator ROJ COLO 3- Cell - 3
Generator ROJ COLO 3- Cell - 4
Generator ROJ COLO 4- Cell - 2


...

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: MSFT Deliery -UPM - ROJ - COLO3- Cell-1
------------------------------------------------------------------------------------------
MSFT Deliery -UPM - ROJ - COLO3- Cell-1
MSFT Deliery -UPM - ROJ - COLO-4 Cell- 2
MSFT Deliery -UPM - ROJ - COLO-4 Cell- 3
MSFT Deliery -UPM - ROJ - COLO-4 Cell- 4
MSFT Deliery -PDU ROJ COLO-1 Cell-1
MSFT Deliery -PDU ROJ COLO-1 Cell-2
MSFT Deliery -PDU ROJ COLO-1 Cell-3
MSFT Deliery -PDU ROJ COLO-1 Cell-4
MSFT Deliery -PDU ROJ COLO-2 Cell-1
MSFT Deliery -PDU ROJ COLO-2 Cell-2
MSFT Deliery -PDU ROJ COLO-2 Cell-3
MSFT Deliery -PDU ROJ COLO-2 Cell-4
MSFT Deliery -PDU ROJ COLO-3 Cell-1
MSFT Deliery -PDU ROJ COLO-3 Cell-2
MSFT Deliery -PDU ROJ COLO-3 Cell-3
MSFT Deliery -PDU ROJ COLO-3 Cell-4
MSFT Deliery -PDU ROJ COLO-4 Cell-1
MSFT Deliery -PDU ROJ COLO-4 Cell-2
MSFT Deliery -PDU ROJ COLO-4 Cell-3
MSFT Deliery -PDU ROJ COLO-4 Cell-4
+++++++++++++++++++++++++++++

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: MSFT Deliery -Fluid Cooler - ROJ COLO -1 Cell- 3
------------------------------------------------------------------------------------------
MSFT Deliery -Fluid Cooler - ROJ COLO -1 Cell- 3
MSFT Deliery -Fluid Cooler - ROJ COLO -2 Cell-1
MSFT Deliery -Fluid Cooler - ROJ COLO -2 Cell-2
MSFT Deliery -Fluid Cooler - ROJ COLO -3 Cell-1
MSFT Deliery -Fluid Cooler - ROJ COLO -3 Cell-2
MSFT Deliery -Fluid Cooler - ROJ COLO -3 Cell-4
MSFT Deliery -Fluid Cooler - ROJ COLO -4 Cell-1
MSFT Deliery -Fluid Cooler - ROJ COLO -4 Cell-2
MSFT Deliery -Fluid Cooler - ROJ COLO -4 Cell-4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: MSFT Deliery -Generator ROJ COLO 1- Cell-1
------------------------------------------------------------------------------------------
MSFT Deliery -Generator ROJ COLO 1- Cell-1
MSFT Deliery -Generator ROJ COLO 2- Cell - 2
MSF

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: MSFT Deliery -PDU ROJ COLO-3 Cell-1
------------------------------------------------------------------------------------------
MSFT Deliery -PDU ROJ COLO-3 Cell-1
MSFT Deliery -PDU ROJ COLO-4 Cell-2
MSFT Deliery -PDU ROJ COLO-4 Cell-3
MSFT Deliery -PDU ROJ COLO-4 Cell-4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: MSFT Deliery -PDU ROJ COLO-3 Cell-2
------------------------------------------------------------------------------------------
MSFT Deliery -PDU ROJ COLO-3 Cell-2
MSFT Deliery -PDU ROJ COLO-4 Cell-1
MSFT Deliery -PDU ROJ COLO-4 Cell-3
MSFT Deliery -PDU ROJ COLO-4 Cell-4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: MSFT Deliery -PDU ROJ COLO-3 Cell-3
------------------------------------------------------------------------------------------
MSFT Deliery -PDU ROJ COLO-3

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Duct Bank - Building to North Vault URS run
------------------------------------------------------------------------------------------
Duct Bank - Building to North Vault URS run
Duct Bank - Builing to South Vault- URS Run
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Set MV Gear (URS)
------------------------------------------------------------------------------------------
Set MV Gear (URS)
Set MV Gear (UMS)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: PULL- URS to COLO-1
------------------------------------------------------------------------------------------
PULL- URS to COLO-1
PULL- URS to COLO-2
PULL- URS to COLO-3
PULL- URS to COLO-4
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: PULL- URS to COLO-2
-----------------

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Set Electrical Equipment-UPM
------------------------------------------------------------------------------------------
Set Electrical Equipment-UPM
Set Electrical Equipment-UPM Cell -3
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: EPMS Pull Wire / Term / Test
------------------------------------------------------------------------------------------
EPMS Pull Wire / Term / Test
BMS Pull Wire / Term / Test
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
key name: Install Production Fiber (Includes MOR rack set & Fiber runner)
------------------------------------------------------------------------------------------
Install Production Fiber (Includes MOR rack set & Fiber runner)
Install Production Fiber (Includes MOR rack set)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

## Clusters as features  
Index and store names' clusters  

In [23]:
num = 0 
nums_clusters = []
for cluster_name, cluster in names_clusters.items():
    num += 1
    for name in cluster:
        nums_clusters.append([name, num])
nums_clusters = pd.DataFrame(nums_clusters, columns = [entity, 'distance_cluster'])
num_distance_clusters = len(set(nums_clusters['distance_cluster']))
print('{n} clusters identified'.format(n=num_distance_clusters))
nums_clusters.head(30)

237 clusters identified


,Activity Name,distance_cluster
0,MWH06 Admin/COLO1 - LV3 Cx Complete,1
1,MWH06 COLO2 - LV3 Cx Complete,1
2,MWH06 COLO3 - LV3 Cx Complete,1
3,MWH06 COLO4 - LV3 Cx Complete,1
4,MWH06 Admin/COLO1 - Fitout Summary,2
5,MWH06 COLO2 - Fitout Summary,2
6,MWH06 COLO3 - Fitout Summary,2
7,MWH06 COLO4 - Fitout Summary,2
8,Foundation Start - Admin/COLO 1,3
9,Fit out Start - Admin/COLO 1,3


# Cluster analysis: Activities Features  

## Encode categorical features  

In [24]:
entity_values = list(activity_data[entity])
encoded = one_hot_encode_categorical(activity_data, entity).dropna()
print('encoded dataset rows/columns counts=', encoded.shape)

encoded dataset rows/columns counts= (1115, 3623)


## Dimensionality reduction  

In [25]:
import numpy as np
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X = encoded.drop([entity], axis=1)#.to_numpy()
Xreduced = pca.fit(X)
# summarize components
print("Explained Variance: %s" % Xreduced.explained_variance_ratio_)
components = Xreduced.components_.T
print('components shape:', components.shape)

Explained Variance: [0.91946286 0.04809676]
components shape: (3622, 2)


## Cluster samples: KMeans  

In [44]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=50)
kmeans.fit(components)
features_labels = kmeans.predict(components)
clusters_counts = pd.Series(features_labels).value_counts()
clusters_counts = pd.DataFrame(list(zip(clusters_counts.index, clusters_counts.values)),\
                              columns = ['cluster', 'count']).sort_values(by=['count'], ascending=False)
clusters_counts.to_excel('samples_per_kmeans_cluster_50_clusters.xlsx', index=False)
print('Samples per Cluster:')
clusters_counts[:10]

Samples per Cluster:


,cluster,count
0,0,891
1,44,544
2,27,540
3,48,502
4,20,464
5,32,178
6,42,159
7,22,89
8,26,68
9,17,36


# Compare Clusters  

In [49]:
features_labels_df = pd.DataFrame(list(zip(entity_values, features_labels))\
                                  , columns = [entity, 'kmeans_cluster'])
clusters_df = pd.merge(nums_clusters, features_labels_df, how='left').\
sort_values(by='kmeans_cluster')
clusters_df.to_excel('clusters_comparison.xlsx', index=False)
clusters_df.head(50)

,Activity Name,distance_cluster,kmeans_cluster
2441,Level 3 Cx - COLO-4,237,0
1551,MSFT Deliery -PDU ROJ COLO-1 Cell-1,141,0
1550,MSFT Deliery -Generator ROJ COLO 1- Cell-3,141,0
1536,MSFT Deliery -PDU ROJ COLO-4 Cell-4,140,0
1535,MSFT Deliery -PDU ROJ COLO-4 Cell-3,140,0
1534,MSFT Deliery -PDU ROJ COLO-4 Cell-2,140,0
1533,MSFT Deliery -PDU ROJ COLO-4 Cell-1,140,0
1532,MSFT Deliery -PDU ROJ COLO-3 Cell-4,140,0
1531,MSFT Deliery -PDU ROJ COLO-3 Cell-3,140,0
1530,MSFT Deliery -PDU ROJ COLO-3 Cell-2,140,0
